In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

To start, let's assign the dataframe to df.

In [ ]:
df = pd.read_csv("/kaggle/input/reccendata/RecCen_Fall2022.csv")
print(df.head(20))

As we can see, the first row of the dataframe isn't necessary. Furthermore, the columns are not properly aligned. Let's fix this.

In [ ]:
df['Date'], df['Visits'] = df['Date'].shift(), df['Visits'].shift() #correctly align columns
df = df.drop(df.index[[0,1]]).reset_index(drop=True) #remove invalid rows
print(df.head(20))

In order to more efficiently process and visualize our data, let's convert the values in the 'Date' and 'Time' columns to datetime objects. Let's also convert the values in 'Time' into their corresponding values on the 24-hour clock, and create an additional column called 'Datetime' that combines the values in 'Date' with the newly revised values in 'Time.'

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M %p').dt.time
df['Time'] = df['Time'].apply(lambda x: x.strftime('%H:%M'))
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))
print(df.head(20))

Now, let's remove any hours on days of operation during which the rec center was closed. This will help us better visualized how crowded it was each day depending not only on how many people swiped in but also how long the rec center was open.

In [ ]:
df = df[~(df['Datetime'] < '2022-09-18 12:00:00')]
df = df[~df['Datetime'].between('2022-11-23 15:00:00', '2022-11-23 22:00:00')]
df = df[~(df['Datetime'] > '2022-12-10 19:00:00')]
df = df[~((df['Day'].isin(['Saturday', 'Sunday'])) & (df['Time'] < '09:00'))]
df = df[~((df['Day'].isin(['Friday', 'Saturday'])) & (df['Time'] > '20:00'))]
df = df[~((df['Day'] == 'Sunday') & (df['Time'] > '21:00'))]
print(df.head(20))

It's time for our first visualization.

In [ ]:
# Plot the data with color-coded bars
visits_per_hour = df.groupby('Date')['Visits'].sum() / df.groupby('Date').size()
plt.figure(figsize=(10, 6))
visits_per_hour.plot(kind='bar', color="skyblue")
plt.title('Mean Visits per Hour of Operation Each Day')
plt.xlabel('Date')
plt.ylabel('Mean Visits per Hour')
plt.xticks(range(0, len(visits_per_hour), 7), visits_per_hour.index[::7].strftime('%m/%d'), rotation=45)
plt.tight_layout()
plt.show()

For the following visualizations, let's also remove days when the rec center was closed.

In [ ]:
df = df[~(df['Date'] == '2022-11-11')]
df = df[~df['Date'].between('2022-11-24', '2022-11-27')]

First, let's plot the mean and standard deviation of visits to the rec center across hours of the day and day of the week.

In [ ]:
mean_visits = df.groupby([df['Datetime'].dt.hour, 'Day'])['Visits'].mean().unstack()
std_visits = df.groupby([df['Datetime'].dt.hour, 'Day'])['Visits'].std().unstack()
plt.figure(figsize=(15, 6))
days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
for i, day in enumerate(days_of_week):
    x_values = mean_visits.index + i * 0.15 - 0.3 # Adjust the offset here
    plt.errorbar(x_values, mean_visits[day], yerr=std_visits[day], fmt='o', label=day)
plt.title('Mean and Standard Deviation of Visits Based on Hour and Day of the Week')
plt.xlabel('Hour')
plt.ylabel('Visits')
plt.xticks(np.arange(6, 23))
plt.legend(title='Day')
plt.grid(True)
plt.tight_layout()
plt.show()

We can also create a heatmap to visualize the mean visits across hours and days of the week.

In [ ]:
df['Day'] = pd.Categorical(df['Day'], categories=days_of_week, ordered=True)
pivot_df = df.pivot_table(index='Time', columns='Day', values='Visits', aggfunc='mean', observed=False)
plt.figure(figsize=(12, 6))
plt.imshow(pivot_df, cmap='plasma', aspect='auto', interpolation='nearest')
plt.colorbar(label='Mean Visits')
plt.xticks(range(len(pivot_df.columns)), pivot_df.columns)
plt.yticks(range(len(pivot_df.index)), pivot_df.index)
plt.xlabel('Day')
plt.ylabel('Hour')
plt.title('Mean Visits Based on Hour and Day of the Week')
plt.tight_layout()
plt.show()